In [1]:
import pandas as pd
import numpy as np

# Load datasets
fundamentals = pd.read_csv("fundamentals.csv")
prices = pd.read_csv("prices-split-adjusted.csv")
securities = pd.read_csv("securities.csv")

In [3]:
# Ensure latest data for each company in fundamentals based on Period Ending
fundamentals['date'] = pd.to_datetime(fundamentals['date'])
fundamentals = fundamentals.sort_values('date').drop_duplicates(subset='Ticker Symbol', keep='last')

# Ensure latest stock price for each company in prices based on date
prices['date'] = pd.to_datetime(prices['date'])
latest_prices = prices.groupby('symbol')['date'].max().reset_index()  # Get the latest date for each symbol
latest_prices = pd.merge(prices, latest_prices, on=['symbol', 'date'], how='inner')  # Merge to keep only latest rows

# Merge fundamentals with latest stock prices
merged_df = fundamentals.merge(latest_prices[['symbol', 'close']], left_on='Ticker Symbol', right_on='symbol', how='inner')

# Compute Market Cap (Market Cap = Shares Outstanding * Close Price)
merged_df["Market Cap"] = merged_df["Estimated Shares Outstanding"] * merged_df["close"]

# Calculate Financial Ratios
merged_df['D/E Ratio'] = merged_df['Total Liabilities'] / merged_df['Total Equity']
merged_df['P/E Ratio'] = merged_df["Market Cap"] / merged_df['Earnings Per Share']
merged_df.dropna(inplace=True)


C:\Users\Nikkhil_Bisht\AppData\Local\Temp\ipykernel_22124\3179632717.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fundamentals['date'] = pd.to_datetime(fundamentals['date'])


In [4]:
# Normalize Values
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

merged_df['W_EPS'] = normalize(merged_df['Earnings Per Share'])
merged_df['W_PE'] = 1 - normalize(merged_df['P/E Ratio'])  # Lower P/E is better
merged_df['W_DE'] = 1 - normalize(merged_df['D/E Ratio'])  # Lower D/E is better

# Simulated CIBIL Score (as API integration is required)
merged_df['W_CIBIL'] = np.random.uniform(700, 900, merged_df.shape[0]) / 900  # Normalize to [0,1]

# Compute Final Weight Score
merged_df['Index Score'] = (
    0.35 * normalize(merged_df['Market Cap']) +
    0.20 * merged_df['W_CIBIL'] +
    0.15 * merged_df['W_EPS'] +
    0.15 * merged_df['W_PE'] +
    0.15 * merged_df['W_DE']
)

In [6]:
# Select top n companies
n = 10
top_n_companies = merged_df.nlargest(n, 'Index Score')
print("Top", n, "Companies by Index Score:\n", top_n_companies[['Ticker Symbol', 'Index Score']])


Top 10 Companies by Index Score:
     Ticker Symbol  Index Score
311          AAPL     0.750633
290          MSFT     0.627764
179           XOM     0.568177
257           CRM     0.513099
258           WMT     0.507622
122           CVX     0.502424
253            HD     0.497522
102             T     0.493175
172          PCLN     0.483059
114            VZ     0.479481


In [7]:
# Calculate CAGR for each company in the top N companies
def calculate_cagr(ticker):
    # Get the historical stock prices for the company
    company_data = prices[prices['symbol'] == ticker].sort_values(by='date')
    
    # Get initial and final values
    initial_price = company_data.iloc[0]['close']
    final_price = company_data.iloc[-1]['close']
    
    # Calculate the number of years between initial and final period
    initial_year = pd.to_datetime(company_data.iloc[0]['date']).year
    final_year = pd.to_datetime(company_data.iloc[-1]['date']).year
    num_years = final_year - initial_year
    
    # Calculate CAGR
    if num_years > 0:
        cagr = (final_price / initial_price) ** (1 / num_years) - 1
    else:
        cagr = 0  # If no years, assume no growth
    
    return cagr

In [8]:
# Get the top N companies
top_n_prices = latest_prices[latest_prices['symbol'].isin(top_n_companies['Ticker Symbol'])]

# Simulate future prices using CAGR for each top N company
years = 1  # 1-year projection
top_n_prices['CAGR'] = top_n_prices['symbol'].apply(calculate_cagr)

# Simulate future price for each company using its own CAGR
top_n_prices['Future Price'] = top_n_prices['close'] * (1 + top_n_prices['CAGR']) ** years

# Mutual Fund Profit Estimation
mutual_fund_value_now = top_n_prices['close'].mean()
mutual_fund_value_future = top_n_prices['Future Price'].mean()
expected_return = ((mutual_fund_value_future / mutual_fund_value_now) - 1) * 100

print(f"\nExpected Mutual Fund Return in {years} year(s): {expected_return:.2f}%")



Expected Mutual Fund Return in 1 year(s): 29.48%


C:\Users\Nikkhil_Bisht\AppData\Local\Temp\ipykernel_22124\149396554.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_n_prices['CAGR'] = top_n_prices['symbol'].apply(calculate_cagr)
C:\Users\Nikkhil_Bisht\AppData\Local\Temp\ipykernel_22124\149396554.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_n_prices['Future Price'] = top_n_prices['close'] * (1 + top_n_prices['CAGR']) ** years
